In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [4]:
daily_act = pd.read_csv('./dailyActivity_merged.csv')
sleep_day = pd.read_csv('./sleepDay_merged.csv')
sleep_min = pd.read_csv('./minuteSleep_merged.csv')
step_min = pd.read_csv('./minuteStepsNarrow_merged.csv')
intens_min = pd.read_csv('./minuteIntensitiesNarrow_merged.csv')
heart_rate = pd.read_csv('./heartrate_seconds_merged.csv')
weight = pd.read_csv('./weightLogInfo_merged.csv')
calories_min = pd.read_csv('./minuteCaloriesNarrow_merged.csv')

In [5]:
daily_act = pd.read_csv('./dailyActivity_merged.csv')
sleep_day = pd.read_csv('./sleepDay_merged.csv')
sleep_min = pd.read_csv('./minuteSleep_merged.csv')
step_min = pd.read_csv('./minuteStepsNarrow_merged.csv')
intens_min = pd.read_csv('./minuteIntensitiesNarrow_merged.csv')
heart_rate = pd.read_csv('./heartrate_seconds_merged.csv')
weight = pd.read_csv('./weightLogInfo_merged.csv')
calories_min = pd.read_csv('./minuteCaloriesNarrow_merged.csv')

In [6]:
distinct_daily_act = daily_act['Id'].nunique()
distinct_sleep_day = sleep_day['Id'].nunique()
distinct_sleep_min = sleep_min['Id'].nunique()
distinct_step_min = step_min['Id'].nunique()
distinct_intens_min = intens_min['Id'].nunique()
distinct_heart_rate = heart_rate['Id'].nunique()
distinct_weight = weight['Id'].nunique()
distinct_calories_min = calories_min['Id'].nunique()


In [7]:
duplicates_daily_act = daily_act.duplicated(subset=['Id']).sum()
duplicates_sleep_day = sleep_day.duplicated(subset=['Id']).sum()
duplicates_sleep_min = sleep_min.duplicated(subset=['Id']).sum()
duplicates_step_min = step_min.duplicated(subset=['Id']).sum()
duplicates_intens_min = intens_min.duplicated(subset=['Id']).sum()
duplicates_heart_rate = heart_rate.duplicated(subset=['Id']).sum()
duplicates_weight = weight.duplicated(subset=['Id']).sum()
duplicates_calories_min = calories_min.duplicated(subset=['Id']).sum()

In [8]:
sleep_day = sleep_day.drop_duplicates(subset=['Id'])
sleep_min = sleep_min.drop_duplicates(subset=['Id'])

In [9]:
Id_days = daily_act.groupby('Id').size().reset_index(name='days')

Id_days['category'] = pd.cut(Id_days['days'],
                             bins=[0, 10, 20, 31],
                             labels=["low use", "moderate use", "high use"])

Id_days_new = Id_days.groupby('category').size().reset_index(name='user_number')

print(Id_days_new.head())

       category  user_number
0       low use            1
1  moderate use            3
2      high use           29


In [17]:
daily_act['ActivityDate'] = pd.to_datetime(daily_act['ActivityDate'], format='%m/%d/%Y')

# Print the column names of sleep_day to ensure 'Id' is present
print("Column names of sleep_day:", sleep_day.columns)

# Separate 'SleepDay' column in sleep_day
sleep_day1 = sleep_day[['SleepDay']].copy()
sleep_day1[['ActivityDate', 'time', 'am_pm']] = sleep_day1['SleepDay'].str.split(' ', expand=True)
sleep_day1['ActivityDate'] = pd.to_datetime(sleep_day1['ActivityDate'], format='%m/%d/%Y')

# Merge 'Id' column from daily_act into sleep_day1
sleep_day1 = pd.merge(sleep_day1, daily_act[['Id', 'ActivityDate']], on='ActivityDate', how='left')

# Merge two tables
full_day = pd.merge(daily_act, sleep_day1, on=['Id', 'ActivityDate'], how='outer')

# Fill NaN values with 0
full_day = full_day.fillna(0)

# Print the results
print(full_day.head())

Column names of sleep_day: Index(['Id', 'SleepDay', 'TotalSleepRecords', 'TotalMinutesAsleep',
       'TotalTimeInBed'],
      dtype='object')
           Id ActivityDate  TotalSteps  TotalDistance  TrackerDistance  \
0  1503960366   2016-04-12       13162            8.5              8.5   
1  1503960366   2016-04-12       13162            8.5              8.5   
2  1503960366   2016-04-12       13162            8.5              8.5   
3  1503960366   2016-04-12       13162            8.5              8.5   
4  1503960366   2016-04-12       13162            8.5              8.5   

   LoggedActivitiesDistance  VeryActiveDistance  ModeratelyActiveDistance  \
0                       0.0                1.88                      0.55   
1                       0.0                1.88                      0.55   
2                       0.0                1.88                      0.55   
3                       0.0                1.88                      0.55   
4                       0.0

In [18]:
sleep_min = sleep_min.copy()

sleep_min['minutes'] = sleep_min['value']

sleep_min.loc[sleep_min['minutes'] != 0, 'minutes'] = 1

print(sleep_min.describe())

                 Id      value         logId  minutes
count  2.400000e+01  24.000000  2.400000e+01     24.0
mean   4.763820e+09   1.875000  1.140184e+10      1.0
std    2.288233e+09   0.797414  3.987637e+07      0.0
min    1.503960e+09   1.000000  1.137223e+10      1.0
25%    2.340408e+09   1.000000  1.137484e+10      1.0
50%    4.501862e+09   2.000000  1.138018e+10      1.0
75%    6.822462e+09   2.250000  1.141987e+10      1.0
max    8.792010e+09   3.000000  1.152362e+10      1.0


In [19]:
import pandas as pd


sleep_min[['SleepDate', 'time', 'am_pm']] = sleep_min['date'].str.split(' ', expand=True)
sleep_min['SleepDate'] = pd.to_datetime(sleep_min['SleepDate'], format='%m/%d/%Y')

sleep_min1 = sleep_min.groupby(['Id', 'SleepDate']).agg(total_sleep=('minutes', 'sum')).reset_index()

full_day_new = pd.merge(full_day, sleep_min1, left_on=['Id', 'ActivityDate'], right_on=['Id', 'SleepDate'], how='outer')

full_day_new = full_day_new.fillna(0)

print(full_day_new.head())


           Id         ActivityDate  TotalSteps  TotalDistance  \
0  1503960366  2016-04-12 00:00:00     13162.0            8.5   
1  1503960366  2016-04-12 00:00:00     13162.0            8.5   
2  1503960366  2016-04-12 00:00:00     13162.0            8.5   
3  1503960366  2016-04-12 00:00:00     13162.0            8.5   
4  1503960366  2016-04-12 00:00:00     13162.0            8.5   

   TrackerDistance  LoggedActivitiesDistance  VeryActiveDistance  \
0              8.5                       0.0                1.88   
1              8.5                       0.0                1.88   
2              8.5                       0.0                1.88   
3              8.5                       0.0                1.88   
4              8.5                       0.0                1.88   

   ModeratelyActiveDistance  LightActiveDistance  SedentaryActiveDistance  \
0                      0.55                 6.06                      0.0   
1                      0.55                 6.